# Optimizing Dask Workloads

In [1]:
import coiled

cluster = coiled.Cluster(n_workers=50, region="us-east-2")  # start workers close to data to minimize costs
client = cluster.get_client()

Output()

Output()

In [2]:
client

<Client: 'tls://10.0.41.70:8786' processes=12 threads=48, memory=178.14 GiB>

# Use Case: Data cleaning and exploration

Reading, formatting, writing dataset. Preparing it for additional downstream analaysis. 

In [28]:
%%time

import dask.dataframe as dd

ddf = dd.read_csv(
    "s3://coiled-datasets/uber-lyft-tlc-sample/csv-10/*", 
    # "s3://coiled-datasets/uber-lyft-tlc-sample/csv-ill/*", 
    dtype={"wav_match_flag": "category"},
)

CPU times: user 1min 5s, sys: 3.22 s, total: 1min 8s
Wall time: 1min 12s


In [29]:
%%time

# Convert to datetime
ddf["on_scene_datetime"] = dd.to_datetime(ddf["on_scene_datetime"], format="mixed")
ddf["pickup_datetime"] = dd.to_datetime(ddf["pickup_datetime"], format="mixed")

# Unpack columns
ddf = ddf.assign(
    accessible_vehicle=ddf.on_scene_datetime.isnull(),
    pickup_month=ddf.pickup_datetime.dt.month,
    pickup_dow=ddf.pickup_datetime.dt.dayofweek,
    pickup_hour=ddf.pickup_datetime.dt.hour,
)
ddf = ddf.drop(columns=["on_scene_datetime", "pickup_datetime"])

# Format airport_fee
ddf["airport_fee"] = ddf["airport_fee"].replace("None", 0).astype(float).fillna(0)

# Remove outliers
lower_bound = 0
Q3 = ddf["trip_time"].quantile(0.75)
upper_bound = Q3 + (1.5 * (Q3 - lower_bound))
ddf = ddf.loc[(ddf["trip_time"] >= lower_bound) & (ddf["trip_time"] <= upper_bound)]

# # Categorize *_flag columns
# ddf = ddf.categorize(columns=["shared_request_flag", "shared_match_flag", "access_a_ride_flag", "wav_request_flag", "wav_match_flag"])

service_names = {
    "HV0002": "juno",
    "HV0005": "lyft",
    "HV0003": "uber",
    "HV0004": "via",
}

ddf["service_names"] = ddf["hvfhs_license_num"].map(service_names)

CPU times: user 139 ms, sys: 7.54 ms, total: 147 ms
Wall time: 146 ms


In [30]:
ddf

,Unnamed: 0,hvfhs_license_num,dispatching_base_num,originating_base_num,request_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,trip_time,base_passenger_fare,tolls,bcf,sales_tax,congestion_surcharge,airport_fee,tips,driver_pay,shared_request_flag,shared_match_flag,access_a_ride_flag,wav_request_flag,wav_match_flag,accessible_vehicle,pickup_month,pickup_dow,pickup_hour,service_names
npartitions=10703,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,int64,object,object,object,object,object,int64,int64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,object,object,object,object,category[unknown],bool,int32,int32,int32,object
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


Do some analysis

In [31]:
%%time

(ddf.tips != 0).mean().compute()

CPU times: user 1.05 s, sys: 100 ms, total: 1.15 s
Wall time: 1min 20s


0.1583987882760626

In [32]:
%%time

ddf.loc[lambda x: x.tips > 0].groupby("service_names").tips.sum().compute()

CPU times: user 920 ms, sys: 83 ms, total: 1 s
Wall time: 1min 12s


service_names
juno    1.053204e+06
lyft    9.243761e+07
uber    1.984338e+08
via     1.486593e+06
Name: tips, dtype: float64

In [33]:
%%time

ddf.loc[lambda x: x.tips > 0].groupby("service_names").tips.mean().compute()

CPU times: user 1.05 s, sys: 103 ms, total: 1.15 s
Wall time: 1min 23s


service_names
juno    3.928619
lyft    4.837245
uber    4.763513
via     2.314891
Name: tips, dtype: float64

# Persist Intelligently 

In [34]:
%%time

ddf = ddf.persist()

CPU times: user 622 ms, sys: 15.4 ms, total: 638 ms
Wall time: 643 ms


In [35]:
%%time

from distributed import wait
wait(ddf);

CPU times: user 1.25 s, sys: 101 ms, total: 1.35 s
Wall time: 1min 9s


In [36]:
%%time

(ddf.tips != 0).mean().compute()

CPU times: user 395 ms, sys: 25 ms, total: 420 ms
Wall time: 16.1 s


0.1583987882760626

In [37]:
%%time

ddf.loc[lambda x: x.tips > 0].groupby("service_names").tips.sum().compute()

CPU times: user 185 ms, sys: 15.9 ms, total: 201 ms
Wall time: 7.23 s


service_names
juno    1.053204e+06
lyft    9.243761e+07
uber    1.984338e+08
via     1.486593e+06
Name: tips, dtype: float64

In [38]:
%%time

ddf.loc[lambda x: x.tips > 0].groupby("service_names").tips.mean().compute()

CPU times: user 318 ms, sys: 27.3 ms, total: 346 ms
Wall time: 17.8 s


service_names
juno    3.928619
lyft    4.837245
uber    4.763513
via     2.314891
Name: tips, dtype: float64

# Compute Intelligently

In [23]:
trip_frac = (ddf.tips != 0).mean()
gb_sum = ddf.loc[lambda x: x.tips > 0].groupby("service_names").tips.sum()
gb_mean = ddf.loc[lambda x: x.tips > 0].groupby("service_names").tips.mean()

In [24]:
%%time

import dask

trip_frac, gb_sum, gb_mean = dask.compute(trip_frac, gb_sum, gb_mean)

CPU times: user 612 ms, sys: 41 ms, total: 653 ms
Wall time: 33.7 s


# File format

In [39]:
%%time

import dask.dataframe as dd

# ddf = dd.read_csv(
#     "s3://coiled-datasets/uber-lyft-tlc-sample/csv-ill/*", 
#     dtype={"wav_match_flag": "category"},
# )

# ddf = dd.read_parquet("s3://coiled-datasets/uber-lyft-tlc-sample/parquet-ill/")
ddf = dd.read_parquet("s3://coiled-datasets/uber-lyft-tlc-sample/parquet-10/")

CPU times: user 1.8 s, sys: 108 ms, total: 1.91 s
Wall time: 3.83 s


In [40]:
ddf.dtypes

hvfhs_license_num       string[python]
dispatching_base_num    string[python]
originating_base_num    string[python]
request_datetime        datetime64[ns]
on_scene_datetime       datetime64[ns]
pickup_datetime         datetime64[ns]
dropoff_datetime        datetime64[ns]
PULocationID                     int32
DOLocationID                     int32
trip_miles                     float32
trip_time                        int32
base_passenger_fare            float32
tolls                          float32
bcf                            float32
sales_tax                      float32
congestion_surcharge           float32
airport_fee                    float32
tips                           float32
driver_pay                     float32
shared_request_flag           category
shared_match_flag             category
access_a_ride_flag            category
wav_request_flag              category
wav_match_flag                category
dtype: object

In [41]:
%%time

# # Convert to datetime
# ddf["on_scene_datetime"] = dd.to_datetime(ddf["on_scene_datetime"], format="mixed")
# ddf["pickup_datetime"] = dd.to_datetime(ddf["pickup_datetime"], format="mixed")

# Unpack columns
ddf = ddf.assign(
    accessible_vehicle=ddf.on_scene_datetime.isnull(),
    pickup_month=ddf.pickup_datetime.dt.month,
    pickup_dow=ddf.pickup_datetime.dt.dayofweek,
    pickup_hour=ddf.pickup_datetime.dt.hour,
)
ddf = ddf.drop(columns=["on_scene_datetime", "pickup_datetime"])

# Format airport_fee
ddf["airport_fee"] = ddf["airport_fee"].replace("None", 0).astype(float).fillna(0)

# Remove outliers
lower_bound = 0
Q3 = ddf["trip_time"].quantile(0.75)
upper_bound = Q3 + (1.5 * (Q3 - lower_bound))
ddf = ddf.loc[(ddf["trip_time"] >= lower_bound) & (ddf["trip_time"] <= upper_bound)]

# # Categorize *_flag columns
# ddf = ddf.categorize(columns=["shared_request_flag", "shared_match_flag", "access_a_ride_flag", "wav_request_flag", "wav_match_flag"])

service_names = {
    "HV0002": "juno",
    "HV0005": "lyft",
    "HV0003": "uber",
    "HV0004": "via",
}

ddf["service_names"] = ddf["hvfhs_license_num"].map(service_names)

CPU times: user 117 ms, sys: 7.48 ms, total: 124 ms
Wall time: 120 ms


In [42]:
ddf = ddf.persist()

In [43]:
%%time

from distributed import wait
wait(ddf);

CPU times: user 1.16 s, sys: 80.9 ms, total: 1.24 s
Wall time: 38.4 s


In [44]:
%%time

(ddf.tips != 0).mean().compute()

CPU times: user 285 ms, sys: 22.6 ms, total: 308 ms
Wall time: 14.8 s


0.1583987882760626

In [45]:
%%time

ddf.loc[lambda x: x.tips > 0].groupby("service_names").tips.sum().compute()

CPU times: user 189 ms, sys: 17.7 ms, total: 206 ms
Wall time: 7.65 s


service_names
juno    1.053204e+06
lyft    9.243762e+07
uber    1.984339e+08
via     1.486593e+06
Name: tips, dtype: float32

In [46]:
%%time

ddf.loc[lambda x: x.tips > 0].groupby("service_names").tips.mean().compute()

CPU times: user 421 ms, sys: 27.1 ms, total: 448 ms
Wall time: 16.6 s


service_names
juno    3.928619
lyft    4.837245
uber    4.763513
via     2.314891
Name: tips, dtype: float64

# Partition size

In [47]:
import dask
dask.utils.format_bytes(ddf.partitions[0].compute().memory_usage(deep=True).sum())

'11.98 MiB'

In [48]:
%%time

ddf = ddf.repartition("100MiB")
ddf = ddf.persist()
wait(ddf);

CPU times: user 3.56 s, sys: 562 ms, total: 4.12 s
Wall time: 28.3 s


In [49]:
dask.utils.format_bytes(ddf.partitions[0].compute().memory_usage(deep=True).sum())

'95.81 MiB'

In [50]:
%%time

(ddf.tips != 0).mean().compute()

CPU times: user 62.5 ms, sys: 7.93 ms, total: 70.5 ms
Wall time: 1.89 s


0.1583987882760626

In [51]:
%%time

ddf.loc[lambda x: x.tips > 0].groupby("service_names").tips.sum().compute()

CPU times: user 53.4 ms, sys: 5.86 ms, total: 59.2 ms
Wall time: 1.33 s


service_names
juno    1.053204e+06
lyft    9.243762e+07
uber    1.984339e+08
via     1.486593e+06
Name: tips, dtype: float32

In [52]:
%%time

ddf.loc[lambda x: x.tips > 0].groupby("service_names").tips.mean().compute()

CPU times: user 65.6 ms, sys: 6.69 ms, total: 72.3 ms
Wall time: 2.93 s


service_names
juno    3.928619
lyft    4.837245
uber    4.763513
via     2.314891
Name: tips, dtype: float64

# Data types

Look at a dashboard plot (maybe the GIL contention plot?) that demostrates we could benefit from PyArrow strings

In [53]:
ddf.dtypes

hvfhs_license_num       string[python]
dispatching_base_num    string[python]
originating_base_num    string[python]
request_datetime        datetime64[ns]
dropoff_datetime        datetime64[ns]
PULocationID                     int32
DOLocationID                     int32
trip_miles                     float32
trip_time                        int32
base_passenger_fare            float32
tolls                          float32
bcf                            float32
sales_tax                      float32
congestion_surcharge           float32
airport_fee                    float64
tips                           float32
driver_pay                     float32
shared_request_flag           category
shared_match_flag             category
access_a_ride_flag            category
wav_request_flag              category
wav_match_flag                category
accessible_vehicle                bool
pickup_month                     int32
pickup_dow                       int32
pickup_hour              

In [54]:
ddf = ddf.astype({
    "service_names": "string[pyarrow]",
    "hvfhs_license_num": "string[pyarrow]",
    "dispatching_base_num": "string[pyarrow]",
    "originating_base_num": "string[pyarrow]",
})

In [55]:
%%time

ddf = ddf.persist()
wait(ddf);

CPU times: user 692 ms, sys: 139 ms, total: 831 ms
Wall time: 3.76 s


In [56]:
ddf.dtypes

hvfhs_license_num       string[pyarrow]
dispatching_base_num    string[pyarrow]
originating_base_num    string[pyarrow]
request_datetime         datetime64[ns]
dropoff_datetime         datetime64[ns]
PULocationID                      int32
DOLocationID                      int32
trip_miles                      float32
trip_time                         int32
base_passenger_fare             float32
tolls                           float32
bcf                             float32
sales_tax                       float32
congestion_surcharge            float32
airport_fee                     float64
tips                            float32
driver_pay                      float32
shared_request_flag            category
shared_match_flag              category
access_a_ride_flag             category
wav_request_flag               category
wav_match_flag                 category
accessible_vehicle                 bool
pickup_month                      int32
pickup_dow                        int32


In [57]:
dask.utils.format_bytes(ddf.partitions[1].compute().memory_usage(deep=True).sum())

'37.06 MiB'

In [72]:
# dask.config.set({"dataframe.convert-string": True});

In [58]:
%%time

ddf = ddf.repartition("100MB")
ddf = ddf.persist()
wait(ddf);

CPU times: user 693 ms, sys: 95.2 ms, total: 788 ms
Wall time: 4.64 s


In [59]:
%%time

(ddf.tips != 0).mean().compute()

CPU times: user 39.4 ms, sys: 4.54 ms, total: 43.9 ms
Wall time: 898 ms


0.1583987882760626

In [60]:
%%time

ddf.loc[lambda x: x.tips > 0].groupby("service_names").tips.sum().compute()

CPU times: user 34.9 ms, sys: 4.87 ms, total: 39.8 ms
Wall time: 1.44 s


service_names
juno    1.053204e+06
lyft    9.243762e+07
uber    1.984338e+08
via     1.486593e+06
Name: tips, dtype: float32

In [61]:
%%time

ddf.loc[lambda x: x.tips > 0].groupby("service_names").tips.mean().compute()

CPU times: user 58.8 ms, sys: 6.67 ms, total: 65.5 ms
Wall time: 1.29 s


service_names
juno    3.928619
lyft    4.837245
uber    4.763513
via     2.314891
Name: tips, dtype: float64

# Conclusions

- Explored Dask dashboard
    - Saw plots that are available
    - Learned to interpret them
- Best practices
    - File format, partition size, data types, etc.